# Import libraries

In [1]:
# !pip install ccxt 
# !pip install pandas
# !pip install matplotlib


Defaulting to user installation because normal site-packages is not writeable
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
   ---------------------------------------- 0.0/5.7 MB ? eta -:--:--
   ---------------- ----------------------- 2.4/5.7 MB 12.2 MB/s eta 0:00:01
   -------------------------------------- - 5.5/5.7 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 5.7/5.7 MB 12.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   ---------------------------------------- 3.2/3.2 MB 27.0 MB/s eta 0:00:00
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 1.2/1.2 MB 20.6 MB/s eta 0:00:00
Using cached idna-3.10-py3-none-any.whl (70 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Load API keys 

# Load exchange 

# Function for executing trades

Testing

In [ ]:
import ccxt
import time
import pandas as pd
from datetime import datetime

# Binance API keys (Replace with your actual API keys)
BINANCE_API_KEY = 'your_api_key'
BINANCE_SECRET_KEY = 'your_secret_key'

# Initialize Binance Futures API
binance = ccxt.binance({
    'apiKey': BINANCE_API_KEY,
    'secret': BINANCE_SECRET_KEY,
    'options': {'defaultType': 'future'}
})

# Settings
SYMBOL = 'BTC/USDT'
LEVERAGE = 5
TRADE_AMOUNT = 0.01  # Adjust based on risk
FUNDING_THRESHOLD = 0.0005  # Arbitrage trigger threshold
DATA_FILE = "funding_arbitrage_data.csv"

# Store trade history
columns = ["timestamp", "funding_rate", "position", "profit", "balance"]
df = pd.DataFrame(columns=columns)


# Function to fetch funding rate
def get_funding_rate(symbol):
    try:
        funding_rate = binance.fetch_funding_rate(symbol)
        return funding_rate['fundingRate']
    except Exception as e:
        print(f"Error fetching funding rate: {e}")
        return None


# Function to place trades
def place_order(symbol, side, amount, leverage=LEVERAGE):
    try:
        binance.set_leverage(leverage, symbol)
        order = binance.create_market_order(symbol, side, amount)
        print(f"Order placed: {side} {amount} {symbol}")
        return order
    except Exception as e:
        print(f"Error placing order: {e}")
        return None


# Function to close positions
def close_positions(symbol, amount):
    place_order(symbol, "sell", amount)  # Close long position
    place_order(symbol, "buy", amount)  # Close short position


# Main strategy function
def funding_arbitrage():
    global df

    funding_rate = get_funding_rate(SYMBOL)
    if funding_rate is None:
        return

    print(f"Current funding rate: {funding_rate}")

    if funding_rate > FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: LONG position opened.")
        order = place_order(SYMBOL, "buy", TRADE_AMOUNT)
        position = "long"
    elif funding_rate < -FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: SHORT position opened.")
        order = place_order(SYMBOL, "sell", TRADE_AMOUNT)
        position = "short"
    else:
        print("No arbitrage opportunity found.")
        return

    # Wait for funding period (8 hours)
    time.sleep(8 * 60 * 60)

    # Close the trade
    close_positions(SYMBOL, TRADE_AMOUNT)

    # Log trade data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    profit = TRADE_AMOUNT * funding_rate * 8  # Approximate funding profit
    balance = binance.fetch_balance()['total']['USDT']

    new_entry = pd.DataFrame([[timestamp, funding_rate, position, profit, balance]], columns=columns)
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save to CSV for later comparison
    df.to_csv(DATA_FILE, index=False)
    print("Trade logged successfully.\n")


# Run the script periodically (Every 8 hours)
while True:
    funding_arbitrage()


To check if api key can connect to binance

In [28]:

import ccxt

# ✅ Replace with your Binance API keys
api_key = 'cYvphVwGXGAid79K7sE6rYfDRQepPehoFX9VAUEe8TLpV3l4VHTQ8fVZsTOHEQBy'
api_secret = 'qFQ5pYAVfI6LhqnPz3hSRKNuMt56Z5b2Hlt1u8xFMI9kORQdpgSz7NSTKYmHylFC'

binance = ccxt.binance({
    'apiKey': api_key,
    'secret': api_secret,
    'enableRateLimit': True,
    'options': {'defaultType': 'future'}
})

try:
    # Test fetching balance
    balance = binance.fetch_balance()
    print("✅ Futures API Connection Successful! USDT Balance:", balance['total']['USDT'])
except Exception as e:
    print("⛔ API Connection Failed:", e)



✅ Futures API Connection Successful! USDT Balance: 0.0


To check if bianance future api is online

In [8]:
import requests

url = "https://fapi.binance.com/fapi/v1/ping"
response = requests.get(url)

if response.status_code == 200:
    print("✅ Binance Futures API is online!")
else:
    print(f"⛔ Binance Futures API is down! Status code: {response.status_code}")


✅ Binance Futures API is online!


## Funding rate trading bot

In [31]:
import ccxt

binance_real = ccxt.binance({
    'apiKey': 'cYvphVwGXGAid79K7sE6rYfDRQepPehoFX9VAUEe8TLpV3l4VHTQ8fVZsTOHEQBy',
    'secret': 'qFQ5pYAVfI6LhqnPz3hSRKNuMt56Z5b2Hlt1u8xFMI9kORQdpgSz7NSTKYmHylFC',
    'options': {'defaultType': 'future'}
})

symbol = "BTC/USDT"  # ✅ Ensure it's in this format for CCXT

try:
    funding_info = binance_real.fetch_funding_rate(symbol)
    print("✅ Funding Rate:", funding_info)
except Exception as e:
    print("❌ Funding Rate Fetch Failed:", e)


✅ Funding Rate: {'info': {'symbol': 'BTCUSDT', 'markPrice': '97454.50000000', 'indexPrice': '97496.30600000', 'estimatedSettlePrice': '97400.45862336', 'lastFundingRate': '0.00008081', 'interestRate': '0.00010000', 'nextFundingTime': '1739203200000', 'time': '1739200515000'}, 'symbol': 'BTC/USDT:USDT', 'markPrice': 97454.5, 'indexPrice': 97496.306, 'interestRate': 0.0001, 'estimatedSettlePrice': 97400.45862336, 'timestamp': 1739200515000, 'datetime': '2025-02-10T15:15:15.000Z', 'fundingRate': 8.081e-05, 'fundingTimestamp': 1739203200000, 'fundingDatetime': '2025-02-10T16:00:00.000Z', 'nextFundingRate': None, 'nextFundingTimestamp': None, 'nextFundingDatetime': None, 'previousFundingRate': None, 'previousFundingTimestamp': None, 'previousFundingDatetime': None, 'interval': None}


In [32]:
binance_testnet = ccxt.binance({
    'apiKey': 'XakkBEsWhJkOkFd2oHV3bN4wf6epSsFVWJ0jYCRkHyjpcEAbQXK0IGiQf2hJj6sA',
    'secret': '4Xy4w8kGORt86tkgupybOHoHyzhx2TSaVZECByBT5bVbYsDxaIHzOZpcrZyAQjT7',
    'options': {'defaultType': 'future'},
    'urls': {'api': 'https://testnet.binancefuture.com'}
})

try:
    funding_info = binance_testnet.fetch_funding_rate(symbol)
    print("✅ Testnet Funding Rate:", funding_info)
except Exception as e:
    print("❌ Testnet Funding Rate Fetch Failed:", e)


❌ Testnet Funding Rate Fetch Failed: binance does not have a testnet/sandbox URL for sapi endpoints


In [47]:
import ccxt
import time
import pandas as pd
from datetime import datetime

# ✅ Real Binance API (For fetching funding rates)
binance_real = ccxt.binance({
    'apiKey': 'cYvphVwGXGAid79K7sE6rYfDRQepPehoFX9VAUEe8TLpV3l4VHTQ8fVZsTOHEQBy',
    'secret': 'qFQ5pYAVfI6LhqnPz3hSRKNuMt56Z5b2Hlt1u8xFMI9kORQdpgSz7NSTKYmHylFC',
    'options': {'defaultType': 'future'}
})

# ✅ Binance Testnet API (For simulated trading)
binance_testnet = ccxt.binance({
    'apiKey': 'mHzjOwXDgA6Lw5vwOxJssvBYGCbqqRBtgTcK2m4KnZo0nxM1rGumWOvEXKWPlZWh',  
    'secret': 'j3ST4Q3i07OfdtBwbiLTMggN35bSlmDBLB4q4saYpKSgibjxqsWDGuBLWMEdPYqC',  
    'options': {'defaultType': 'future'},
    'urls': {
        'api': {
            'public': 'https://testnet.binancefuture.com/fapi/v1',
            'private': 'https://testnet.binancefuture.com/fapi/v1',
        }
    }
})

# Settings
SYMBOL = 'BTC/USDT'
LEVERAGE = 5
TRADE_AMOUNT = 0.01  # Adjust based on risk
FUNDING_THRESHOLD = -0.00007852  # Arbitrage trigger threshold
DATA_FILE = "funding_arbitrage_data_testnet.csv"

# Store trade history
columns = ["timestamp", "funding_rate", "position", "profit", "balance"]
df = pd.DataFrame(columns=columns)

# ✅ Function to fetch funding rate from real Binance API
def get_funding_rate(symbol):
    try:
        funding_info = binance_real.fetch_funding_rate(symbol)
        return funding_info['fundingRate']
    except Exception as e:
        print(f"Error fetching funding rate: {e}")
        return None

# ✅ Function to place trades on Binance Testnet
def place_order(symbol, side, amount, leverage=LEVERAGE):
    try:
        # Set leverage
        binance_testnet.set_leverage(leverage, symbol)
        print(f"Leverage set to {leverage}x for {symbol}")

        # Place market order
        order = binance_testnet.create_order(symbol, 'market', side, amount)
        print(f"Testnet Order placed: {side} {amount} {symbol}")
        return order
    except Exception as e:
        print(f"Error placing testnet order: {e}")
        return None

# ✅ Function to close positions on Binance Testnet
def close_positions(symbol, amount):
    try:
        # Fetch current positions
        positions = binance_testnet.fetch_positions(symbol)
        for position in positions:
            if position['symbol'] == symbol.replace('/', ''):
                if position['side'] == 'long':
                    place_order(symbol, "sell", amount)  # Close long position
                elif position['side'] == 'short':
                    place_order(symbol, "buy", amount)  # Close short position
    except Exception as e:
        print(f"Error closing positions: {e}")

# ✅ Main strategy function
def funding_arbitrage():
    global df

    funding_rate = get_funding_rate(SYMBOL)
    if funding_rate is None:
        return

    print(f"Current Funding Rate: {funding_rate}")

    if funding_rate > FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: LONG position opened.")
        order = place_order(SYMBOL, "buy", TRADE_AMOUNT)
        position = "long"
    elif funding_rate < -FUNDING_THRESHOLD:
        print("Arbitrage opportunity detected: SHORT position opened.")
        order = place_order(SYMBOL, "sell", TRADE_AMOUNT)
        position = "short"
    else:
        print("No arbitrage opportunity found.")
        return

    # Wait for funding period (8 hours)
    time.sleep(8 * 60 * 60)

    # Close the trade
    close_positions(SYMBOL, TRADE_AMOUNT)

    # Log trade data
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    profit = TRADE_AMOUNT * funding_rate * 8  # Approximate funding profit
    balance = binance_testnet.fetch_balance()['total']['USDT']

    new_entry = pd.DataFrame([[timestamp, funding_rate, position, profit, balance]], columns=columns)
    df = pd.concat([df, new_entry], ignore_index=True)

    # Save to CSV for later comparison
    df.to_csv('data.csv', index=False)  # index=False removes the extra index column
    print("Trade logged successfully.\n")

# ✅ Run the script periodically (Every 8 hours)
while True:
    funding_arbitrage()

Current Funding Rate: 7.476e-05
Arbitrage opportunity detected: LONG position opened.
Error placing testnet order: binance {"code":-2008,"msg":"Invalid Api-Key ID."}


KeyboardInterrupt: 

In [ ]:
import ccxt
import time
import pandas as pd
from datetime import datetime

#Real Binance API (For trading and funding rates)
binance = ccxt.binance({
    'apiKey': '',
    'secret': '',
    'options': {'defaultType': 'future'}
})

#Settings
SYMBOL = 'BTC/USDT'
LEVERAGE = 5
TRADE_AMOUNT = 0.01  # Fixed trade size
STOP_LOSS = 95  # Stop-loss price
TAKE_PROFIT = 110  # Take-profit price
TRAILING_STOP_PERCENT = 0.10  # 10% below highest price
FUNDING_THRESHOLD = 0.00008  # Arbitrage trigger threshold
DATA_FILE = "funding_arbitrage_trades.csv"

#Store trade history
columns = ["timestamp", "funding_rate", "position", "entry_price", "exit_price", "profit"]
df = pd.DataFrame(columns=columns)

#Function to fetch funding rate
def get_funding_rate(symbol):
    try:
        funding_rate = binance.fetch_funding_rate(symbol)
        return funding_rate['fundingRate']
    except Exception as e:
        print(f"Error fetching funding rate: {e}")
        return None
#Function to place trades
def place_order(symbol, side, amount):
    try:
        binance.set_leverage(LEVERAGE, symbol)
        order = binance.create_market_order(symbol, side, amount)
        print(f"Order placed: {side} {amount} {symbol}")
        return order['price']
    except Exception as e:
        print(f"Error placing order: {e}")
        return None

#Function to monitor trade and set stop-loss/take-profit
def monitor_trade(position, entry_price):
    highest_price = entry_price  # Track highest price for trailing stop
    while True:
        ticker = binance.fetch_ticker(SYMBOL)
        current_price = ticker['last']
        print(f"Current Price: {current_price}")

#Update highest price for trailing stop
        if current_price > highest_price:
            highest_price = current_price

        # Check stop-loss and take-profit
        if current_price <= STOP_LOSS:
            print("Stop-loss hit! Closing trade.")
            return current_price
        elif current_price >= TAKE_PROFIT:
            print("Take-profit hit! Closing trade.")
            return current_price
        elif current_price <= highest_price * (1 - TRAILING_STOP_PERCENT):
            print("Trailing stop-loss hit! Closing trade.")
            return current_price

        time.sleep(10)  # Check price every 10 seconds
#Main strategy function
def funding_arbitrage():
    global df

    funding_rate = get_funding_rate(SYMBOL)
    if funding_rate is None:
        return

    print(f"Current Funding Rate: {funding_rate}")

    if funding_rate > FUNDING_THRESHOLD:
        print("LONG position opened.")
        entry_price = place_order(SYMBOL, "buy", TRADE_AMOUNT)
        position = "long"
    elif funding_rate < -FUNDING_THRESHOLD:
        print("SHORT position opened.")
        entry_price = place_order(SYMBOL, "sell", TRADE_AMOUNT)
        position = "short"
    else:
        print("No arbitrage opportunity found.")
        return

    if entry_price:
        exit_price = monitor_trade(position, entry_price)
        profit = (exit_price - entry_price) * TRADE_AMOUNT if position == "long" else (entry_price - exit_price) * TRADE_AMOUNT

#Log trade data
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        new_entry = pd.DataFrame([[timestamp, funding_rate, position, entry_price, exit_price, profit]], columns=columns)
        df = pd.concat([df, new_entry], ignore_index=True)
        df.to_csv('data_file_arbitrage,csv', index=False)
        print("Trade logged successfully.")

#Run the script periodically (Every 8 hours)
while True:
    funding_arbitrage()
    time.sleep(8 * 60 * 60)  # Wait 8 hours before next funding check


Current Funding Rate: 8.659e-05
LONG position opened.
Error placing order: binance {"code":-2019,"msg":"Margin is insufficient."}


KeyboardInterrupt: 

In [48]:
import ccxt

binance_testnet = ccxt.binance({
    'apiKey': 'mHzjOwXDgA6Lw5vwOxJssvBYGCbqqRBtgTcK2m4KnZo0nxM1rGumWOvEXKWPlZWh',
    'secret': 'j3ST4Q3i07OfdtBwbiLTMggN35bSlmDBLB4q4saYpKSgibjxqsWDGuBLWMEdPYqC',
    'options': {'defaultType': 'future'},
    'urls': {
        'api': {
            'public': 'https://testnet.binancefuture.com/fapi/v1',
            'private': 'https://testnet.binancefuture.com/fapi/v1',
        }
    }
})

try:
    balance = binance_testnet.fetch_balance()
    print("Balance:", balance)
except Exception as e:
    print("Error:", e)

Error: binance {"code":-2008,"msg":"Invalid Api-Key ID."}


# Back testing portion


In [2]:
!pip install pandas
!pip install matplotlib

  Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached pandas-2.2.3-cp313-cp313-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.2-cp313-cp313-win_amd64.whl (12.6 MB)
  Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------- -------------------------- 2.6/8.0 MB 12.6 MB/s eta 0:00:01
   ----------------------------------- ---- 7.1/8.0 MB 17.4 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 17.1 MB/s eta 0:00:00
Using cached contourpy-1.3.1-cp313-cp313-win_amd64.whl (220 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 26.3 MB/s eta 0:00:00
   -------------------------------

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load historical funding rate data (CSV from Binance API or past logs)
# Ensure the CSV has a "timestamp" and "funding_rate" column
DATA_FILE = "historical_funding_rates.csv" # change 
df = pd.read_csv(DATA_FILE)

# Settings for backtesting
TRADE_AMOUNT = 0.01  # Amount per trade
FUNDING_THRESHOLD = 0.0005  # Arbitrage trigger threshold
LEVERAGE = 5

# Add columns for backtesting results
df["position"] = "none"
df["profit"] = 0

# Simulate trading based on past funding rates
for i in range(len(df) - 1):
    funding_rate = df.loc[i, "funding_rate"]

    if funding_rate > FUNDING_THRESHOLD:
        df.loc[i, "position"] = "long"
        df.loc[i, "profit"] = TRADE_AMOUNT * funding_rate * 8  # Funding profit estimate
    elif funding_rate < -FUNDING_THRESHOLD:
        df.loc[i, "position"] = "short"
        df.loc[i, "profit"] = TRADE_AMOUNT * abs(funding_rate) * 8  # Funding profit estimate
    else:
        df.loc[i, "position"] = "none"

# Calculate total profit
total_profit = df["profit"].sum()
print(f"Total Estimated Profit from Backtest: ${total_profit:.2f}")

# Plot Funding Rate vs. Profit
plt.figure(figsize=(12, 5))
plt.plot(df["timestamp"], df["funding_rate"], label="Funding Rate", color="blue")
plt.axhline(y=FUNDING_THRESHOLD, color="green", linestyle="--", label="Long Threshold")
plt.axhline(y=-FUNDING_THRESHOLD, color="red", linestyle="--", label="Short Threshold")
plt.legend()
plt.title("Funding Rate Over Time")
plt.xlabel("Timestamp")
plt.ylabel("Funding Rate")
plt.xticks(rotation=45)
plt.show()

# Save results for later analysis
df.to_csv("funding_arbitrage_backtest_results.csv", index=False)
